In [ ]:
#One hidden layer neural network. Code from scratch

from sklearn.datasets import fetch_openml
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

X, y = fetch_openml('Fashion-MNIST', version=1, return_X_y=True)
X = X / 255

digits = 10
points = y.shape[0]
print(points)

y = y.reshape(1, points)

Y_new = np.eye(digits)[y.astype('int32')]
Y_new = Y_new.T.reshape(digits, points)

maxi = 60000
max_t = X.shape[0] - maxi

X_train, X_test = X[:maxi].T, X[maxi:].T
Y_train, Y_test = Y_new[:,:maxi], Y_new[:,maxi:]

shuffle_index = np.random.permutation(maxi)
X_train, Y_train = X_train[:, shuffle_index], Y_train[:, shuffle_index]

i = 12
plt.imshow(X_train[:,i].reshape(28,28), cmap = matplotlib.cm.binary)
plt.axis("off")
plt.show()
print(Y_train[:,i])

def cross_entropy_loss(Y, Y_hat):

    L_sum = np.sum(np.multiply(Y, np.log(Y_hat)))
    m = Y.shape[1]
    L = -(1/m) * L_sum

    return L

def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

num_x = X_train.shape[0]
num_h = 64
learning_rate = 0.5

W1 = np.random.randn(num_h, num_x)
b1 = np.zeros((num_h, 1))
W2 = np.random.randn(digits, num_h)
b2 = np.zeros((digits, 1))

X = X_train
Y = Y_train

losstrack = list()

for i in range(1000):

    Z1 = np.matmul(W1,X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.matmul(W2,A1) + b2
    A2 = np.exp(Z2) / np.sum(np.exp(Z2), axis=0)

    cost = cross_entropy_loss(Y, A2)
    losstrack.append(np.squeeze(cost))

    dZ2 = A2-Y
    dW2 = (1./maxi) * np.matmul(dZ2, A1.T)
    db2 = (1./maxi) * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.matmul(W2.T, dZ2)
    dZ1 = dA1 * sigmoid(Z1) * (1 - sigmoid(Z1))
    dW1 = (1./maxi) * np.matmul(dZ1, X.T)
    db1 = (1./maxi) * np.sum(dZ1, axis=1, keepdims=True)

    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1

    if (i % 100 == 0):
        print("Epoch", i, "cost: ", cost)

print("Final cost:", cost)

plt.plot(losstrack)
plt.show()

Z1 = np.matmul(W1, X_test) + b1
A1 = sigmoid(Z1)
Z2 = np.matmul(W2, A1) + b2
A2 = np.exp(Z2) / np.sum(np.exp(Z2), axis=0)

predictions = np.argmax(A2, axis=0)
labels = np.argmax(Y_test, axis=0)
predictions = np.argmax(A2, axis=0)
labels = np.argmax(Y_test, axis=0)

print(confusion_matrix(predictions, labels))
print(classification_report(predictions, labels))

In [ ]:
#NN Using Tensorflow. 
import keras
import tensorflow as tf
from keras.datasets import fashion_mnist
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from keras import backend
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score

mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model_1 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation=tf.nn.tanh),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model_1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


#model.fit(x_train, y_train, epochs=2)
history = model_1.fit(x_train, y_train, epochs=15)
plt.plot(history.history['loss'])

test_predictions = model_1.predict(x_test)

print(y_test.shape)
print(x_test.shape)
# Compute confusion matrix
#confusion = tf.confusion_matrix(y_test, test_predictions)
test_predictions = model_1.predict_classes(x_test)

from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_true = y_test, y_pred = test_predictions)
print(confusion_matrix)
print(classification_report(y_true = y_test, y_pred = test_predictions))

In [ ]:
#NN Using Keras with one hidden layer of 64 Nodes. 
import keras
from keras.datasets import fashion_mnist
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

(X_train,Y_train), (X_test,Y_test) = fashion_mnist.load_data()

X_train = X_train.reshape(-1, 28,28, 1)
X_test = X_test.reshape(-1, 28,28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255
X_test = X_test / 255

Y_train_t_h = to_categorical(Y_train)
Y_test_t_h = to_categorical(Y_test)

model = Sequential()

model.add(Conv2D(64, (3,3), input_shape=(28, 28, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

history = model.fit(X_train, Y_train_t_h, batch_size=64, epochs=5)

plt.plot(history.history['loss'])

test_loss, test_acc = model.evaluate(X_test, Y_test_t_h)
print('Test loss', test_loss)
print('Test accuracy', test_acc)

test_predictions = model.predict_classes(X_test)

from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_true = Y_test, y_pred = test_predictions)
print(confusion_matrix)
print(classification_report(y_true = Y_test, y_pred = test_predictions))  